# Compras Públicas em TIC

In [1]:
import pandas as pd
pd.options.display.float_format = "{:_.0f}".format 

In [2]:
%reload_ext sql

Tip: You may define configurations in /Users/fredguth/Code/mgi/pipelines/pyproject.toml or /Users/fredguth/.jupysql/config.

Did not find user configurations in /Users/fredguth/Code/mgi/pipelines/pyproject.toml.

In [3]:
%config SqlMagic.style = 'DEFAULT'   # or 'DEFAULT', 'MARKDOWN', etc.
%config SqlMagic.autopandas = True         # optional: return DataFrames
%config SqlMagic.feedback = True
%config SqlMagic.named_parameters="disabled"
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 800)

## Inputs
Baixei do Databricks as bases parquet:
- vw_dm_material_item_silver.parquet: base do catmat
- vw_dm_servico_item_silver.parquet: base do catser
- db_prd__vw_ft_pncp_compra_item.parquet: view de itens de compras do pncp no db prd da api_v2 do pncp
- db_prd__vw_ft_pncp_compra.parquet: view de compras do pncp no db prd da api_v2 do pncp

!!! Importante: Conforme foi explicado dia 25/08/2025 pelo Stefano e Guilherme, base **db_prd__vw_ft_pncp_...** só tem dados do PNCP que vem do sistema Compras do governo federal, há outras 250+ fontes possíveis que não estão incluídas.  O sistema Compras é usado por pequena parcela de estados e municípios e na totalidade pelo governo federal.

In [ ]:
rm ../data/inputs/mgi/mgi.db

In [ ]:
!ls ../data/inputs/mgi

catmat_catser.parquet                  scn.parquet
contratacoes_pncp                      selecao_compra_item.parquet
db_prd__vw_ft_pncp_compra_item.parquet vw_dm_material_item_silver.parquet
db_prd__vw_ft_pncp_compra.parquet      vw_dm_servico_item_silver.parquet
de_ncm_para_scn128.parquet


In [4]:
%sql duckdb:///../data/inputs/mgi/mgi__2.db

Connecting to 'duckdb:///../data/inputs/mgi/mgi__2.db'

### Importando os dados brutos para o DuckDB

In [5]:
%%sql
CREATE OR REPLACE TABLE catmat as SELECT DISTINCT * FROM READ_PARQUET('../data/inputs/mgi/vw_dm_material_item_silver.parquet');
CREATE OR REPLACE TABLE catser as SELECT DISTINCT * FROM READ_PARQUET('../data/inputs/mgi/vw_dm_servico_item_silver.parquet');
CREATE OR REPLACE TABLE pncp_compras as SELECT DISTINCT * FROM READ_PARQUET ('../data/inputs/mgi/db_prd__vw_ft_pncp_compra.parquet');
CREATE OR REPLACE TABLE pncp_itens as SELECT DISTINCT * FROM READ_PARQUET ('../data/inputs/mgi/db_prd__vw_ft_pncp_compra_item.parquet');

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


## Itens de Compras de TIC no PNCP

In [6]:
%%sql
DROP VIEW IF EXISTS itens_tic;
CREATE VIEW itens_tic AS
SELECT *
FROM pncp_itens
WHERE tem_resultado
  AND (
    codigo_classe // 100 = 70
    OR codigo_grupo IN (111,112,113,115,131,162,163,164,165,166,168,171,172,173,174,182)
  )
GROUP BY ALL;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,Success


In [7]:
%%sql
SELECT ano_compra, valor_total_homologado FROM pncp_compras limit 2

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,ano_compra,valor_total_homologado
0,2023,16_920
1,2022,47_800


## Normalizando os dados de itens de compra

Os itens de compra tem um cod_item_catalogo que pode ser do CATMAT ou CATSER.  Os códigos do CATMAT sao divididos em `grupos` e `classes` e os do CATSER em `divisão` e `grupos`.  


In [8]:
%%sql
    
;CREATE OR REPLACE VIEW itens_tic__normalizados AS
WITH catalogo AS (
  SELECT
    i.ano_compra,
    i.cod_compra,
    i.cod_item_compra,
    i.ID_contratacao_PNCP AS id_contratacao_pncp,
    i.cod_fornecedor,
    i.nome_fornecedor,
    i.material_ou_servico,
    CASE
      WHEN i.codigo_classe IS NOT NULL THEN m.codigo_grupo
      WHEN i.codigo_grupo  IS NOT NULL THEN s.codigo_divisao
      ELSE NULL
    END AS codigo_grupo_ou_divisao,
    CASE
      WHEN i.codigo_classe IS NOT NULL THEN COALESCE(m.nome_grupo,   '-- desconhecido --')
      WHEN i.codigo_grupo  IS NOT NULL THEN COALESCE(s.nome_divisao, '-- desconhecido --')
      ELSE '-- desconhecido --'
    END AS nome_grupo_ou_divisao,
    CASE
      WHEN i.codigo_classe IS NOT NULL THEN i.codigo_classe
      WHEN i.codigo_grupo  IS NOT NULL THEN i.codigo_grupo::INT
      ELSE NULL
    END AS codigo_classe_ou_grupo,
    CASE
      WHEN i.codigo_classe IS NOT NULL THEN COALESCE(m.nome_classe, '-- desconhecido --')
      WHEN i.codigo_grupo  IS NOT NULL THEN COALESCE(s.nome_grupo,  '-- desconhecido --')
      ELSE '-- desconhecido --'
    END AS nome_classe_ou_grupo,
    i.cod_item_catalogo,
    i.descricao,
    i.valor_total_resultado
  FROM itens_tic i
  LEFT JOIN (
    SELECT
      codigo_classe,
      MIN(nome_classe)  AS nome_classe,
      MIN(codigo_grupo) AS codigo_grupo,
      MIN(nome_grupo)   AS nome_grupo
    FROM catmat
    GROUP BY codigo_classe
  ) m ON i.codigo_classe::INT = m.codigo_classe::INT
  LEFT JOIN (
    SELECT
      codigo_grupo,
      MIN(nome_grupo)     AS nome_grupo,
      MIN(codigo_divisao) AS codigo_divisao,
      MIN(nome_divisao)   AS nome_divisao
    FROM catser
    GROUP BY codigo_grupo
  ) s ON i.codigo_grupo::INT = s.codigo_grupo::INT
),
per_group AS (
  SELECT
    ano_compra, cod_compra,
    cod_item_compra, cod_fornecedor,
    nome_fornecedor,
    material_ou_servico,
    codigo_grupo_ou_divisao,
    nome_grupo_ou_divisao,
    codigo_classe_ou_grupo,
    nome_classe_ou_grupo,
    SUM(valor_total_resultado) AS valor_total_resultado
  FROM catalogo
  GROUP BY ALL
),
ranked AS (
  SELECT
    p.*,
    ROW_NUMBER() OVER (
      PARTITION BY p.cod_item_compra
      ORDER BY p.valor_total_resultado DESC
    ) AS rn
  FROM per_group p
)
SELECT DISTINCT
  ano_compra, cod_compra, cod_item_compra, cod_fornecedor,
  nome_fornecedor, material_ou_servico,
  codigo_grupo_ou_divisao, nome_grupo_ou_divisao,codigo_classe_ou_grupo,
  nome_classe_ou_grupo, valor_total_resultado
FROM ranked
WHERE rn = 1;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,Success


In [9]:
%%sql
SELECT
  COUNT(cod_item_compra)                  AS rows_count,
  COUNT(DISTINCT cod_item_compra)         AS distinct_coditem_compra,
  COUNT(*) = COUNT(DISTINCT cod_item_compra) AS counts_match
FROM itens_tic__normalizados;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,rows_count,distinct_coditem_compra,counts_match
0,101122,101122,True


In [10]:
%%sql
FROM itens_tic__normalizados limit 3

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,ano_compra,cod_compra,cod_item_compra,cod_fornecedor,nome_fornecedor,material_ou_servico,codigo_grupo_ou_divisao,nome_grupo_ou_divisao,codigo_classe_ou_grupo,nome_classe_ou_grupo,valor_total_resultado
0,2021,3566,13421,33134920000151,CARLOS EDUARDO DE OLIVA ANDRADE 83044825504,M,70,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC",7060,PEÇAS E ACESSÓRIOS PARA COMPUTADORES,680
1,2021,17928,396121,21308808000100,A.F.H. COMERCIO E SERVICOS TECNICOS EIRELI,M,70,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC",7080,"PEÇAS, ACESSÓRIOS E FERRAMENTAS PARA REDES DE TIC",12_470
2,2021,18046,398063,07483598000166,GTMAX TECNOLOGIA EM ELETRONICA LTDA,M,70,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC",7020,IMPRESSORAS,3_672


Uma outra questão é que precisamos do valor_homologado, que é da `compra` e não do `item`, mas uma compra pode ter vários itens. Por isso, na normalização, adotamos um `codigo_classe_ou_grupo` representante da compra. 

Mas uma dúvida fica:
- Uma compra tem itens muito diferentes ou agrupando por `codigo_grupo_ou_divisao` e `codigo_classe_ou_grupo` os itens de uma compra estão numa mesma categoria?

Vamos ver:

## 

In [13]:
%%sql
CREATE OR REPLACE VIEW TEMP AS (WITH base AS (
  SELECT
    cod_compra,
    cod_item_compra,
    material_ou_servico,
    nome_classe_ou_grupo AS classe
  FROM itens_tic__normalizados
),
tot AS (
  SELECT cod_compra, COUNT(DISTINCT cod_item_compra) AS n_itens
  FROM base GROUP BY ALL
),
por_classe AS (
  SELECT
    cod_compra,
    material_ou_servico,
    classe,
    COUNT(DISTINCT cod_item_compra) AS n_itens_classe
  FROM base
  GROUP BY ALL
)
SELECT DISTINCT
  p.cod_compra, p.material_ou_servico,
  p.classe,
  p.n_itens_classe,
  t.n_itens,
  ROUND(100.0 * p.n_itens_classe / NULLIF(t.n_itens, 0), 2) AS "classe_%"
FROM por_classe p
JOIN tot t USING (cod_compra)
ORDER BY p.cod_compra, p.n_itens_classe DESC);
SELECT * FROM TEMP ORDER BY "classe_%" desc;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,cod_compra,material_ou_servico,classe,n_itens_classe,n_itens,classe_%
0,6810070,M,EQUIPAMENTOS DE ENTRADA DE DADOS,1,1,100
1,6810084,M,COMPUTADORES,1,1,100
2,6810123,S,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,1,1,100
3,6810156,S,SERVIÇOS DE CONSULTORIA EM TECNOLOGIA DA INFORMAÇÃO ECOMUNICAÇÃO (TIC),1,1,100
4,6810164,S,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,3,3,100
...,...,...,...,...,...,...
46088,7516733,M,"PEÇAS, ACESSÓRIOS E FERRAMENTAS PARA REDES DE TIC",1,113,1
46089,7516733,M,EQUIPAMENTOS DE ENTRADA DE DADOS,1,113,1
46090,7562211,M,EQUIPAMENTOS DE REDE DE TIC - LOCAL E REMOTA,1,116,1
46091,5647319,S,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,1,152,1


In [14]:
%%sql
SELECT * FROM (SUMMARIZE TEMP)

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,cod_compra,BIGINT,10,9162794,41446,6328160.4552969,2196702.366708751,5628972,6718523,7570651,46093,0
1,material_ou_servico,VARCHAR,M,S,2,None,None,None,None,None,46093,0
2,classe,VARCHAR,ARTIGOS DIVERSOS PARA PROCESSAMENTO AUTOMÁTICO DE DADOS,UNIDADES PARA MANUSEIO DE CARTÕES,29,None,None,None,None,None,46093,0
3,n_itens_classe,BIGINT,1,930,83,2.193868917189161,6.068538768999759,1,1,2,46093,0
4,n_itens,BIGINT,1,1025,101,5.133881500444753,13.556105764513319,1,2,5,46093,0
5,classe_%,DOUBLE,0.49,100.0,562,73.5707669277331,33.864257594070715,46.84050775737045,100.0,100.0,46093,0


A média de representatividade é ~75%, ok.

## Compras no PNCP

Vamos olhar as compras no PNCP.

In [15]:
%%sql
SELECT ano_compra, (sum(valor_total_homologado)//1_000_000_000) as homologado_em_bilhoes from pncp_compras group by all order by all

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,ano_compra,homologado_em_bilhoes
0,2021,1
1,2022,9
2,2023,124
3,2024,3_807
4,2025,3_023
5,2026,5


### Todas as Compras válidas

In [16]:
%%sql

CREATE OR REPLACE VIEW compras__validas AS (
  SELECT DISTINCT
    cod_compra,
    ano_compra,
    orgao_entidade_razao_social AS orgao_entidade,
    orgao_entidade_poder_id     AS poder,
    orgao_entidade_esfera_id    AS esfera,
    unidade_orgao_uf_sigla      AS uf,
    unidade_orgao_municipio_nome AS municipio,
    TRIM(objeto_compra) AS objeto,
    valor_total_homologado
  FROM pncp_compras
  WHERE situacao_compra_nome = 'Divulgada no PNCP'
    AND NOT contratacao_excluida
);

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,Success


In [17]:
%%sql
SELECT ano_compra, (sum(valor_total_homologado)//1_000_000_000) as homologado_em_bilhoes from compras__validas group by all order by all

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,ano_compra,homologado_em_bilhoes
0,2021,1
1,2022,9
2,2023,122
3,2024,521
4,2025,222
5,2026,5


## Compras TIC no PNCP

In [18]:
%%sql
-- Any duplicates on the purchase table?
SELECT cod_compra, COUNT(*) AS n
FROM compras__validas
GROUP BY cod_compra
HAVING COUNT(*) > 1
ORDER BY n DESC
LIMIT 10;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,cod_compra,n


In [19]:
%%sql
CREATE OR REPLACE TABLE compras_tic AS
WITH items AS (
  SELECT
    TRY_CAST(cod_compra AS BIGINT)          AS cod_compra,
    material_ou_servico,
    codigo_grupo_ou_divisao,
    nome_grupo_ou_divisao,
    codigo_classe_ou_grupo,
    nome_classe_ou_grupo,
    nome_fornecedor,
    valor_total_resultado
  FROM itens_tic__normalizados
),
top_item AS (
  SELECT
    items.*,
    ROW_NUMBER() OVER (
      PARTITION BY cod_compra
      ORDER BY valor_total_resultado DESC
    ) AS rn
  FROM items
),
top_per_compra AS (
  SELECT
    cod_compra,
    material_ou_servico,
    codigo_grupo_ou_divisao,
    nome_grupo_ou_divisao,
    codigo_classe_ou_grupo,
    nome_classe_ou_grupo,
    nome_fornecedor                         AS fornecedor_top,
    valor_total_resultado                   AS top_item_valor
  FROM top_item
  WHERE rn = 1
),
agg AS (
  SELECT
    cod_compra,
    COUNT(*)                                 AS n_itens_tic,
    COUNT(DISTINCT nome_fornecedor)          AS n_fornecedores,
    COALESCE(SUM(valor_total_resultado), 0)  AS total_resultado_tic
  FROM items
  GROUP BY cod_compra
)
SELECT
  v.ano_compra,
  v.cod_compra,
  v.orgao_entidade,
  v.poder,
  v.esfera,
  v.uf,
  v.municipio,
  v.objeto,
  t.material_ou_servico,
  t.codigo_grupo_ou_divisao,
  t.nome_grupo_ou_divisao,
  t.codigo_classe_ou_grupo,
  t.nome_classe_ou_grupo,
  t.fornecedor_top,
  t.top_item_valor,
  a.n_itens_tic,
  a.n_fornecedores,
  a.total_resultado_tic as valor_total_resultado,
  v.valor_total_homologado,
FROM compras__validas v
JOIN top_per_compra t
  ON TRY_CAST(v.cod_compra AS BIGINT) = t.cod_compra
LEFT JOIN agg a
  ON TRY_CAST(v.cod_compra AS BIGINT) = a.cod_compra;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,Success


Checando se join correto

In [20]:
%%sql
SELECT
  COUNT(cod_compra)                  AS rows_count,
  COUNT(DISTINCT cod_compra)         AS distinct_cod_compra,
  COUNT(*) = COUNT(DISTINCT cod_compra) AS counts_match
FROM compras_tic;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,rows_count,distinct_cod_compra,counts_match
0,33666,33666,True


In [21]:
%%sql
SELECT * FROM compras_tic LIMIT 2;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,ano_compra,cod_compra,orgao_entidade,poder,esfera,uf,municipio,objeto,material_ou_servico,codigo_grupo_ou_divisao,nome_grupo_ou_divisao,codigo_classe_ou_grupo,nome_classe_ou_grupo,fornecedor_top,top_item_valor,n_itens_tic,n_fornecedores,valor_total_resultado,valor_total_homologado
0,2024,7593380,SECRETARIA MUNICIPAL DE DESENVOLVIMENTO ECONOMICO E TRABALHO,E,M,SP,SÃO PAULO,"Prestação de serviços de Comunicação de Dados – SD-WAN, visando atender as necessidades para a Secretaria Municipal de Desenvolvimento Econômico e Trabalho – SMDET",S,16,SERVIÇOS PARA A INFRAESTRUTURA DE TECNOLOGIA DA INFORMAÇÃOE COMUNICAÇÃO (TIC),166,SERVIÇOS DE MANUTENÇÃO E INSTALAÇÃO DE EQUIPAMENTOS DE TIC,EMPRESA DE TECNOLOGIA DA INFORMACAO E COMUNICACAO DO MUNICIPIO DE SAO PAULO - PR,237_010,1,1,237_010,237_010
1,2024,5751987,MUNICIPIO DE CAXIAS DO SUL,N,M,RS,CAXIAS DO SUL,Fornecimento de materiais de expediente.,M,70,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC",7060,PEÇAS E ACESSÓRIOS PARA COMPUTADORES,I9 COMERCIO DE SUPRIMENTOS DE INFORMATICA LTDA,1_182,2,2,1_687,18_508


In [23]:
%%sql
SELECT ano_compra, material_ou_servico,(sum(valor_total_homologado)//1_000_000_000) as homologado_em_bilhoes from compras_tic group by all order by all

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,ano_compra,material_ou_servico,homologado_em_bilhoes
0,2021,M,0
1,2021,S,0
2,2022,M,0
3,2022,S,2
4,2023,M,3
5,2023,S,13
6,2024,M,8
7,2024,S,16
8,2025,M,4
9,2025,S,8


# Executivo Federal em 2024

Olhando só as Executivo Federal em 2024

In [26]:
%%sql
SELECT
  material_ou_servico,
  -- nome_grupo_ou_divisao,
  -- nome_classe_ou_grupo,
  SUM(valor_total_homologado) / 1000000.0 AS total_homologado__em_MM
FROM compras_tic
WHERE ano_compra = 2024
  AND poder = 'E'
  AND esfera = 'F'
GROUP BY ALL
  -- material_ou_servico, nome_grupo_ou_divisao, nome_classe_ou_grupo
HAVING SUM(valor_total_homologado) / 1000000.0 > 0
ORDER BY total_homologado__em_MM DESC;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,material_ou_servico,total_homologado__em_MM
0,S,6_825
1,M,2_670


In [28]:
%%sql
SELECT
  material_ou_servico,
  nome_grupo_ou_divisao,
  nome_classe_ou_grupo,
  SUM(valor_total_homologado) / 1000000.0 AS total_homologado__em_MM
FROM compras_tic
WHERE ano_compra = 2024
  AND poder = 'E'
  AND esfera = 'F'
GROUP BY ALL
  -- material_ou_servico, nome_grupo_ou_divisao, nome_classe_ou_grupo
HAVING SUM(valor_total_homologado) / 1000000.0 > 1
ORDER BY total_homologado__em_MM DESC;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,material_ou_servico,nome_grupo_ou_divisao,nome_classe_ou_grupo,total_homologado__em_MM
0,S,"SERVICOS DE ARRENDAMENTO, LICENCIAMENTO DE DIREITOS E TRANS-FERENCIA DE TECNOLOGIA",SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,2_259
1,S,SERVIÇOS PARA A INFRAESTRUTURA DE TECNOLOGIA DA INFORMAÇÃOE COMUNICAÇÃO (TIC),SERVIÇOS DE HOSPEDAGEM EM TECNOLOGIA DA INFORMAÇÃO E COMUNICACÃO (TIC),1_030
2,M,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC",COMPUTADORES,1_023
3,S,SERVIÇOS PARA A INFRAESTRUTURA DE TECNOLOGIA DA INFORMAÇÃOE COMUNICAÇÃO (TIC),SERVIÇOS AUXILIARES DE TECNOLOGIA DA INFORMAÇÃO E COMUNICAÇÃO (TIC),985
4,S,SERVIÇOS DE COMPUTAÇÃO EM NUVEM,SERVIÇOS DE COMPUTAÇÃO EM NUVEM,795
5,S,SERVIÇOS PARA A INFRAESTRUTURA DE TECNOLOGIA DA INFORMAÇÃOE COMUNICAÇÃO (TIC),SERVIÇOS DE GERENCIAMENTO EM TECNOLOGIA DA INFORMAÇÃO E COMUNICAÇÃO (TIC),683
6,M,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC",EQUIPAMENTOS DE REDE DE TIC - LOCAL E REMOTA,458
7,S,"SERVIÇOS DE PESQUISA, ANÁLISE DE DADOS E INDICADORES, CONSULTORIA E PROJETOS DE TIC",SERVIÇOS DE ANÁLISE DA DADOS E INDICADORES DE TIC,426
8,M,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC",PEÇAS E ACESSÓRIOS PARA COMPUTADORES,251
9,M,"INFORMÁTICA - EQUIPAMENTOS, PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC",SUPRIMENTOS DE INFORMÁTICA - TIC,246


Olhando a lista completa

### SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA	

In [29]:
%%sql
SELECT cod_compra, fornecedor_top as fornecedor, orgao_entidade, poder, esfera, nome_classe_ou_grupo, objeto, (valor_total_homologado//1_000_000) as total_homologado_MM  FROM compras_tic where ano_compra = 2024 and poder = 'E' and esfera = 'F' and nome_classe_ou_grupo ilike '%TRANSFERÊNCIA DE TECNOLOGIA%' order by valor_total_homologado desc limit 5;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,cod_compra,fornecedor,orgao_entidade,poder,esfera,nome_classe_ou_grupo,objeto,total_homologado_MM
0,7412986,BRASOFTWARE INFORMATICA LTDA,MINISTERIO DA GESTAO E DA INOVACAO EM SERVICOS PUBLICOS,E,F,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,"Registro de Preços para contratação de empresa especializada para o fornecimento de licenças de softwares, aplicativos e sistemas operacionais Microsoft, destinados aos usuários finais, à camada cliente/servidor e banco de dados, incluindo serviços e consultoria técnica para implantação, projetos e utilização dos produtos, conforme condições e exigências estabelecidas no Edital e seus anexos.",731
1,7107578,SISTTECH TECNOLOGIA EDUCACIONAL COMERCIO E REPRESENTACAO DE PRODUTOS S/A,INSTITUTO FEDERAL DE EDUCACAO CIENCIA E TECNOLOGIA DO ESPIRITO SANTO,E,F,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,"Registro de preços para futura e eventual aquisição de equipamentos para laboratórios de ensino, pesquisa e extensão, compostos de soluções que auxiliam no processo ensino aprendizagem, para os campi do Instituto Federal de Educação, Ciência e Tecnologia do Espírito Santo – Ifes e Órgãos Participantes, conforme condições e exigências do Edital e seus anexos. LINK do PROCESSO SIPAC: https://sipac.ifes.edu.br/public/jsp/processos/processo_detalhado.jsf?id=164428",210
2,6514144,BRASOFTWARE INFORMATICA LTDA,SERVICO FEDERAL DE PROCESSAMENTO DE DADOS (SERPRO),E,F,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,Contratação de direito de uso de subscrição de softwares sob demanda da plataforma Microsoft.,124
3,6319401,MCR SISTEMAS E CONSULTORIA LTDA,MINISTERIO DA GESTAO E DA INOVACAO EM SERVICOS PUBLICOS,E,F,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,"Contratação de licenças de softwares de Design Gráfico, com direito de atualização e suporte, conforme condições e exigências estabelecidas no Edital e em seus anexos.",101
4,7205198,NTSEC SOLUCOES EM TELEINFORMATICA LTDA,SERVICO FEDERAL DE PROCESSAMENTO DE DADOS (SERPRO),E,F,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRANSFERÊNCIA DE TECNOLOGIA,"Contratação de soluções de proteção de redes para os Clientes e Data Centers do Serpro e para Segmentação de suas Redes Locais internas, com garantia técnica, compostas por clusters de firewalls NGFW (Next-Generation Firewall, ou Firewall de Próxima Geração), clusters de virtualização de firewalls NGFW, e clusters de appliances virtuais de firewalls. As soluções devem incluir seus dispositivos, softwares, e as consoles de gerenciamento integrado, mediante criação de ARP.",83


### SERVIÇOS DE HOSPEDAGEM EM TECNOLOGIA DA INFORMAÇÃO E COMUNICACÃO (TIC)

In [31]:
%%sql
SELECT cod_compra, fornecedor_top as fornecedor, orgao_entidade, poder, esfera, nome_classe_ou_grupo, objeto, (valor_total_homologado//1_000_000) as total_homologado_MM  FROM compras_tic where ano_compra = 2024 and esfera = 'F' and poder = 'E' and nome_classe_ou_grupo ilike '%HOSPEDAGEM%' order by valor_total_homologado desc limit 5;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,cod_compra,fornecedor,orgao_entidade,poder,esfera,nome_classe_ou_grupo,objeto,total_homologado_MM
0,6901065,EMPRESA DE TECNOLOGIA E INFORMACOES DA PREVIDENCIA S.A. - DATAPREV,MINISTERIO DO TRABALHO,E,F,SERVIÇOS DE HOSPEDAGEM EM TECNOLOGIA DA INFORMAÇÃO E COMUNICACÃO (TIC),"Contratação da Empresa de Tecnologia e Informações da Previdência (DATAPREV S.A.), por\nmeio de dispensa de licitação, conforme inciso IX, artigo 75 da Lei n.º 14.133/2021, para execução\nde serviços técnicos especializados de tecnologia da informação e comunicações - TIC, buscando\natendimento das necessidades do Ministério do Trabalho e Emprego (MTE).",817
1,6093713,BLACK BOOK EDITORA LTDA,CONSELHO REGIONAL DE MEDICINA DO DISTRITO FEDERAL,E,F,SERVIÇOS DE HOSPEDAGEM EM TECNOLOGIA DA INFORMAÇÃO E COMUNICACÃO (TIC),"Aquisição de licença de uso para aplicativo/funcionalidade de base de evidências médicas, denominado Aplicativo Blackbook, desenvolvido e comercializado pela empresa BLACK BOOK EDITORA LTDA.",117
2,6138454,CIRION TECHNOLOGIES DO BRASIL LTDA,CONSELHO REGIONAL DE ENGENHARIA E AGRONOMIA DO ESTADO DE SAO PAULO,E,F,SERVIÇOS DE HOSPEDAGEM EM TECNOLOGIA DA INFORMAÇÃO E COMUNICACÃO (TIC),"Serviços de infraestrutura de datacenter, com hospedagem privada de sistemas de software e arquivos, com seus respetivos sistemas operacionais, aplicações e bancos de dados, composto por seus equipamentos (hardware), softwares, licenciamento, planejamento, instalação, migração de dados e aplicações, manutenção, comunicação de dados, suporte, operação, treinamento e gerenciamento da solução de Hospedagem.",36
3,5887229,SOLUCTION LOGISTICA E EVENTOS LTDA,"INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNOLOGIA DE SAO PAULO",E,F,SERVIÇOS DE HOSPEDAGEM EM TECNOLOGIA DA INFORMAÇÃO E COMUNICACÃO (TIC),"Registro de preços para eventual contratação de serviço especializado visando a organização e realização de eventos diversos de pequeno, médio e grande porte para o Instituto Federal de Educação, Ciência e Tecnologia de São Paulo – IFSP e demais órgãos participantes.",33
4,7185511,CIRION TECHNOLOGIES DO BRASIL LTDA,SUPERINTENDENCIA DE SEGUROS PRIVADOS,E,F,SERVIÇOS DE HOSPEDAGEM EM TECNOLOGIA DA INFORMAÇÃO E COMUNICACÃO (TIC),"Contratação de empresa especializada para prestação de serviços de hospedagem de aplicações e soluções de TIC em data center da Contratada (hosting), para atender às necessidades da Susep - Superintendência de Seguros Privados, conforme condições e exigências estabelecidas no edital e em seus anexos.",8


### Maiores Fornecedores

In [38]:
%%sql
SELECT
  fornecedor_top AS fornecedor,
  SUM(valor_total_homologado) / 1000000.0 AS total_homologado_MM
FROM compras_tic
WHERE ano_compra = 2024
  AND esfera = 'F'
  AND poder = 'E'
GROUP BY fornecedor_top
ORDER BY total_homologado_MM DESC
LIMIT 10;

Running query in 'duckdb:///../data/inputs/mgi/mgi__2.db'

,fornecedor,total_homologado_MM
0,EMPRESA DE TECNOLOGIA E INFORMACOES DA PREVIDENCIA S.A. - DATAPREV,1_594
1,BRASOFTWARE INFORMATICA LTDA,1_066
2,LASA PROSPECCOES S.A.,359
3,EXTREME DIGITAL EXPERIENCE LTDA.,313
4,SISTTECH TECNOLOGIA EDUCACIONAL COMERCIO E REPRESENTACAO DE PRODUTOS S/A,210
5,ABSOLUT TECHNOLOGIES PROJETOS E CONSULTORIA LTDA,163
6,STEFANINI CONSULTORIA E ASSESSORIA EM INFORMATICA S.A.,157
7,CHAIN TECNOLOGIA E SERVICOS LTDA,130
8,NTSEC SOLUCOES EM TELEINFORMATICA LTDA,125
9,DECISION SERVICOS DE TECNOLOGIA DA INFORMACAO LTDA,124
